https://docs.pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Get Device for Training
# We want to be able to train our model on an accelerator such as CUDA, MPS, MTIA, or XPU. 
# If the current accelerator is available, we will use it. Otherwise, we use the CPU.

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
# Define the Class
# We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. 
# Every nn.Module subclass implements the operations on input data in the forward method.

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),                 # Rectified Linear Unit
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# We create an instance of NeuralNetwork, and move it to the device, and print its structure.

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [4]:
# To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. 
# Do not call model.forward() directly!

# Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted 
# values for each class, and dim=1 corresponding to the individual values of each output. 
# We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [5]:
# Model Layers
# Let’s break down the layers in the FashionMNIST model. 
# To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [6]:
# nn.Flatten
# We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous 
# array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

flatten = nn.Flatten()
flat_image = flatten(input_image)     # This reshapes each image from [28, 28] into [784]
print(flat_image.size())

torch.Size([3, 784])


In [7]:
# nn.Linear
# The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [8]:
# nn.ReLU
# Non-linear activations are what create the complex mappings between the model’s inputs and outputs. 
# They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

# In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

# Applies ReLU activation elementwise:
#   Negative values → 0.
#   Positive values stay.
#   Shape remains [3, 20].

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


Before ReLU: tensor([[-0.1557, -0.0477, -0.3675,  0.2467,  0.0627,  0.3779,  0.2151, -0.0777,
         -0.3403, -0.4098,  0.2680,  0.1332, -0.1248,  0.3162, -0.3490,  0.4881,
          0.1806, -0.5445,  0.1760,  0.2466],
        [-0.1710,  0.1124,  0.1547,  0.3035,  0.2247,  0.3726,  0.2344, -0.2395,
         -0.2020, -0.4775, -0.4473,  0.2667, -0.3255,  0.5978, -0.3558,  0.3728,
          0.1433, -0.3998,  0.3591,  0.2287],
        [-0.4777,  0.5368, -0.5133,  0.6519,  0.1419,  0.3453,  0.0216,  0.1967,
         -0.2563, -0.3045, -0.5791,  0.0743, -0.1868,  0.5448,  0.1124,  0.6200,
          0.0428, -0.2521,  0.4924,  0.2079]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.2467, 0.0627, 0.3779, 0.2151, 0.0000, 0.0000,
         0.0000, 0.2680, 0.1332, 0.0000, 0.3162, 0.0000, 0.4881, 0.1806, 0.0000,
         0.1760, 0.2466],
        [0.0000, 0.1124, 0.1547, 0.3035, 0.2247, 0.3726, 0.2344, 0.0000, 0.0000,
         0.0000, 0.0000, 0.2667, 0.0000, 0.5978, 0.00

In [11]:
# nn.Sequential
# nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. 
# You can use sequential containers to put together a quick network like seq_modules.
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)  
print(input_image.shape)             # torch.Size([3, 28, 28]) - This is like 3 grayscale images of size 28×28.
logits = seq_modules(input_image)    # logits = The raw, unnormalized outputs of a model before they are converted to probabilities. 
print(logits.shape)

torch.Size([3, 28, 28])
torch.Size([3, 10])


In [12]:
# nn.Softmax
# The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. 
# The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. 
# dim parameter indicates the dimension along which the values must sum to 1.

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)


In [13]:
# Model Parameters
# Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. 
# Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters 
# accessible using your model’s parameters() or named_parameters() methods.

# In this example, we iterate over each parameter, and print its size and a preview of its values.

print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0231, -0.0106, -0.0268,  ...,  0.0003,  0.0311,  0.0315],
        [-0.0062,  0.0056,  0.0084,  ...,  0.0010, -0.0021,  0.0082]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0166, -0.0126], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0094, -0.0435,  0.0307,  ..., -0.0407, -0.0278, -0.0248],
        [ 0.0102,  0.0395, -0.0337,  ..., -0.0191,  0.0397, -0.0436]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

In [14]:
import torch
from torch import nn

model = NeuralNetwork()  # your class

# Count params
total = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total params: {total:,} | Trainable: {trainable:,}")

# Per-layer stats (mean/std) without breaking autograd
with torch.no_grad():
    for name, p in model.named_parameters():
        print(f"{name:35s} | shape={tuple(p.shape)} | mean={p.mean():+.4f} | std={p.std():+.4f}")


Total params: 669,706 | Trainable: 669,706
linear_relu_stack.0.weight          | shape=(512, 784) | mean=-0.0000 | std=+0.0206
linear_relu_stack.0.bias            | shape=(512,) | mean=+0.0002 | std=+0.0212
linear_relu_stack.2.weight          | shape=(512, 512) | mean=+0.0000 | std=+0.0255
linear_relu_stack.2.bias            | shape=(512,) | mean=+0.0005 | std=+0.0266
linear_relu_stack.4.weight          | shape=(10, 512) | mean=+0.0008 | std=+0.0253
linear_relu_stack.4.bias            | shape=(10,) | mean=-0.0042 | std=+0.0257
